# Collect Shake Shack Data

# Crawl information of Current Shake Shack Stores

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import unquote

domain = 'https://www.shakeshack.com'
req = requests.get(f'{domain}/locations/')
soup = BeautifulSoup(req.content,"html.parser")

## Collecting Address of New York Subbranches

In [2]:
NYC = soup.find_all("div",{"id":"usa_New_York"})
directions = NYC[0].find_all("a",{"class":"js-dir"})
names = NYC[0].find_all("h4")
locations = []
shopnames = []
for i, direction in enumerate(directions):
    location = direction["data-location"].replace("+"," ")
    location = location.replace("%2C","")
    location = location.replace("%0D%0A","")
    location = location.replace("%C2%A0","")
    location = location.replace("%26","&")
    location = location.replace("%2F","")
    location = location.replace("Shake Shack ","")
    locations.append(location)
    shopnames.append(names[i].get_text())


locations[locations.index('8000 Mall Walk Yonkers NY 10704')]=locations[locations.index('8000 Mall Walk Yonkers NY 10704')].replace("8000","2090")

locations

locations[locations.index('40.753302-74.000577')]=locations[locations.index('40.753302-74.000577')].replace("40.753302-74.000577","20 Hudson Yards New York NY 10001")

## Transform string address into coordinates by geopy

Reference: https://geopy.readthedocs.io/en/stable/ 

In [20]:
from geopy.geocoders import Nominatim, MapBox
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()


mapbox = MapBox(api_key="pk.eyJ1IjoiY2h4dWhlc2giLCJhIjoiY2tya2U1NWUyMjAxMjJ1bzJpZTk5cDEyayJ9.mdXnk3I-E208ofa3i-E5-g")
geocode = RateLimiter(mapbox.geocode, min_delay_seconds=1, max_retries=3)

import pandas as pd
df = pd.DataFrame({'name': shopnames,'location_ShakeShack': locations})

#geolocator = Nominatim(user_agent="myuseragent")

#from geopy.extra.rate_limiter import RateLimiter
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['point'] = df['location_ShakeShack'].apply(geocode).apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude', 'longitude', 'altitude']] =   pd.DataFrame(df['point'].tolist(), index=df.index)


C:\Users\Harmony_Chen\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [116]:
len(df.name.unique())

38

# Searching branches nearby subway through Geopandas

transform `geocoded.csv`，into `geopandas.GeoDataFrame`，set up suitable range for subway area.
Reference: https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.buffer.html
https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.sjoin.html

In [21]:
import geopandas
from shapely.geometry import Point

geocoded = geopandas.read_file("geocoded.csv")


geocoded[geocoded["field_6"]==""]=None
geocoded.dropna(axis=0,subset = ["field_6", "field_7"],inplace=True)

geocoded["geometry"]=geopandas.GeoSeries(list(zip(geocoded["field_6"].astype(dtype="float"),geocoded["field_7"].astype(dtype="float")))).apply(Point)

geocoded.dropna(axis=0,subset = ["geometry"],inplace=True)

geoshakeshack = geopandas.GeoDataFrame(df)

geoshakeshack.rename(columns= {"point":"geometry"},inplace=True)

geoshakeshack["geometry"]=geoshakeshack["geometry"].apply(Point)

ShakeShack_Area = geopandas.GeoDataFrame(df["name"], geometry = geoshakeshack["geometry"].buffer(0.02))
spacial_join_station = geopandas.sjoin(ShakeShack_Area,geocoded,how='left',op='contains') 

ShakeShack_area_station= spacial_join_station[["name","field_1","field_2"]]



C:\Users\Harmony_Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  # Remove the CWD from sys.path while we load stuff.


In [190]:
ShakeShack_area_station.to_csv("ShakeShack_area_station.csv")

# Updating Shake Shack new opening branches

craw from websites like https://investor.shakeshack.com/investors-overview/news/press-release-details/2018/Shake-Shack-Announces-Third-Quarter-2018-Financial-Results/default.aspx


In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import unquote

# each quarter/annual report link
quarterly = 'https://investor.shakeshack.com/investors-overview/news/press-release-details/{year}/Shake-Shack-Announces-{quarter}-Quarter-{year}-Financial-Results/default.aspx'
annual = 'https://investor.shakeshack.com/investors-overview/news/press-release-details/{year_next}/Shake-Shack-Announces-Fourth-Quarter-and-Fiscal-Year-Ended-{year}-Financial-Results/default.aspx'
annual_after_2019 = 'https://investor.shakeshack.com/investors-overview/news/press-release-details/{year_next}/Shake-Shack-Announces-Fourth-Quarter-and-Fiscal-Year-{year}-Financial-Results/default.aspx'


Seperating stores that were open before ShakeShack stock started trading on the stock market (before 2015) from all stores.

Set opening data of those earlier stores as 2014-12-31 uniformly.


In [146]:
#crawl from first three quarter news in each year
Year = range(2015,2022)
Quarter = ["First","Second","Third"]
Local_Record =[]

for year in Year:
    for quarter in Quarter:
        if (year ==2021) and (quarter =="Second"):
            break
        locdomain = (quarterly).format(year=year,quarter=quarter)
        locreq = requests.get(locdomain)
        beausoup = BeautifulSoup(locreq.content,"html.parser")
        ModuleContainerInner = beausoup.find_all("div",{"class":"ModuleContainerInner"})        
        Location = ModuleContainerInner[0].find("b",text = "Location")
        if(Location.parent.name =="p"):
            table = Location.parent.parent.parent.parent
            texts = table.find_all("p")
        else:
            table = Location.parent.parent.parent
            texts = table.find_all("td")
        for text in texts:
            if(text.get_text().find(",")!=-1):
                Local_Record.append([text.get_text().replace("\r","").replace("\n",""),year,quarter])

#crawl forth news in each year before 2019
for year in Year:
    if year == 2019:
        break
    locdomain = (annual).format(year_next=year+1,year=year)
    locreq = requests.get(locdomain)
    beausoup = BeautifulSoup(locreq.content,"html.parser")
    ModuleContainerInner = beausoup.find_all("div",{"class":"ModuleContainerInner"})
    table = ModuleContainerInner[0].find("b",text = "Location").parent.parent.parent
    texts = table.find_all("td")
    for text in texts:
        if(text.get_text().find(",")!=-1):
            Local_Record.append([text.get_text().replace("\r","").replace("\n",""),year,"Fourth"])
            
#crawl forth news in each year starting from 2019
Year_2019 = [2019,2020]
for year in Year_2019:
    locdomain = annual_after_2019.format(year_next=year+1,year=year)
    locreq = requests.get(locdomain)
    beausoup = BeautifulSoup(locreq.content,"html.parser")
    ModuleContainerInner = beausoup.find_all("div",{"class":"ModuleContainerInner"})
    Location = ModuleContainerInner[0].find("b",text = "Location")
    if(Location.parent.name =="p"):
        table = Location.parent.parent.parent.parent
        texts = table.find_all("p")
    else:
        table = Location.parent.parent.parent
        texts = table.find_all("td")
    for text in texts:
        if(text.get_text().find(",")!=-1):
            Local_Record.append([text.get_text().replace("\r","").replace("\n",""),year,"Fourth"])


In [147]:
#export brances data
import pandas
import re
Record = pandas.DataFrame(Local_Record,columns=["Location","Year","Quarter"])
Record["Location"]=Record["Location"].str.strip()
#remove duplicate data
Record.drop_duplicates(subset="Location",inplace=True)
#save brach data
Record.to_csv("Location_after_2015.csv")
Record.drop(Record[Record["Location"]=="Forest Hills, NY — Forest Hills"].index,axis=0,inplace=True)
Record.loc[Record["Location"]=="Forest Hills, NY — Forest HIlls",["Location"]]="Forest Hills, NY — Forest Hills"
#filtrate branches in New York after 2015
content = re.compile('.*, NY.*')
Location_NY_2015 = pd.DataFrame(Record[Record["Location"].str.match(content)])

In [148]:
#New opening stores in NewYork after 2015
len(Location_NY_2015)

24

In [149]:
Location_NY_2015.sort_values(by=['Year','Quarter'],ascending=True)

,Location,Year,Quarter
219,"Central Valley, NY — Woodbury Commons",2015,Fourth
225,"Elmhurst, NY — Queens Center",2015,Fourth
22,"New Hyde Park, NY — Lake Success Shopping Center",2015,Third
253,"New York, NY — Penn Station",2016,Fourth
35,"New York, NY — Herald Square",2016,Second
36,"Forest Hills, NY — Forest Hills",2016,Second
50,"New York, NY — Fulton Transit Center",2016,Third
52,"Yonkers, NY — Cross County Shopping Center",2016,Third
70,"Lake Grove, NY — Smith Haven Plaza",2017,First
261,"New York, NY — Astor Place",2017,Fourth


In [150]:
quarter_dict = {"First":1,"Second":2,"Third":3,"Fourth":4}
def quarter_trans(row):
    return quarter_dict[row["Quarter"]]
Location_NY_2015["Quarter"]=Location_NY_2015.apply(quarter_trans,axis=1)

In [151]:
def year_quarter(row):
    return "{}Q{}".format(row["Year"],row["Quarter"])

Location_NY_2015["QUARTER"]=Location_NY_2015.apply(year_quarter,axis=1)

In [153]:
Location_NY_2015.sort_values("QUARTER")

,Location,Year,Quarter,QUARTER
22,"New Hyde Park, NY — Lake Success Shopping Center",2015,3,2015Q3
219,"Central Valley, NY — Woodbury Commons",2015,4,2015Q4
225,"Elmhurst, NY — Queens Center",2015,4,2015Q4
35,"New York, NY — Herald Square",2016,2,2016Q2
36,"Forest Hills, NY — Forest Hills",2016,2,2016Q2
50,"New York, NY — Fulton Transit Center",2016,3,2016Q3
52,"Yonkers, NY — Cross County Shopping Center",2016,3,2016Q3
253,"New York, NY — Penn Station",2016,4,2016Q4
70,"Lake Grove, NY — Smith Haven Plaza",2017,1,2017Q1
74,"Melville, NY — Melville",2017,2,2017Q2


In [88]:
#Current total number of brances in New York
Location_NY = pd.DataFrame(df["name"])
len(Location_NY)

38

In [157]:
#filtrate New York branches based on opening date before 2015
Location_NY_2015["location"]=Location_NY_2015["Location"].str.lower()
Location_NY_2015["location"]  =Location_NY_2015["Location"].str.split("—").str[-1]
Location_NY_2015["location"]=Location_NY_2015["location"].str.strip()

Location_NY["open_date"]= "2014Q4"
for shop in Location_NY_2015["location"]:
    if Location_NY["name"].str.contains(shop).any():
        date = ("{year}Q{quarter}").format(year=Location_NY_2015.loc[Location_NY_2015["location"]==shop,"Year"].values[0],quarter=Location_NY_2015.loc[Location_NY_2015["location"]==shop,"Quarter"].values[0])
        Location_NY.loc[Location_NY["name"].str.contains(shop),"open_date"]=date


Location_NY.loc[Location_NY["name"]=="Woodbury Common, NY location details","open_date"]="2015Q4"
Location_NY.loc[Location_NY["name"]=="Cross County, NY location details","open_date"]="2016Q3"
Location_NY.loc[Location_NY["name"]=="Lake Grove, Long Island location details","open_date"]="2017Q1"
Location_NY.loc[Location_NY["name"]=="New Hyde Park, Long Island location details","open_date"]="2015Q3"
Location_NY.loc[Location_NY["name"]=="LaGuardia Airport, NYC location details","open_date"]="2018Q4"

In [158]:
#number of stores open before in New York
len(Location_NY[Location_NY["open_date"]=="2014Q4"])

15

In [160]:
Location_NY  =Location_NY.sort_values("open_date")

In [134]:
#current New York branches
Location_NY["name"]

0          1700 Broadway, New York, NY location details
1                     Astor Place, NYC location details
2               Battery Park City, NYC location details
3                 Bay Plaza, Bronx, NY location details
4                     Bryant Park, NYC location details
5                      Citi Field, NYC location details
6                     Cross County, NY location details
7               Downtown Brooklyn, NYC location details
8                 Dumbo, Brooklyn, NYC location details
9     Empire Outlets, Staten Island, NY location det...
10      Flatbush Avenue, Brooklyn, NYC location details
11                    Forest Hills, NY location details
12          Fulton Transit Center, NYC location details
13         Grand Central Terminal, NYC location details
14                          Harlem, NY location details
15                       Hartsdale, NY location details
16                  Herald Square, NYC location details
17                    Hudson Yards, NY location 

In [133]:
list(spacial_join_station["name"].unique())

['1700 Broadway, New York, NY location details',
 'Astor Place, NYC location details',
 'Battery Park City, NYC location details',
 'Bay Plaza, Bronx, NY location details',
 'Bryant Park, NYC location details',
 'Citi Field, NYC location details',
 'Cross County, NY location details',
 'Downtown Brooklyn, NYC location details',
 'Dumbo, Brooklyn, NYC location details',
 'Empire Outlets, Staten Island, NY location details',
 'Flatbush Avenue, Brooklyn, NYC location details',
 'Forest Hills, NY location details',
 'Fulton Transit Center, NYC location details',
 'Grand Central Terminal, NYC location details',
 'Harlem, NY location details',
 'Hartsdale, NY location details',
 'Herald Square, NYC location details',
 'Hudson Yards, NY location details',
 'JFK Airport, NYC location details',
 'LaGuardia Airport, NYC location details',
 'Lake Grove, Long Island location details',
 'Madison Square Park, NYC location details',
 'Melville, Long Island location details',
 'Midtown East, NYC locat

In [161]:
Location_NY.to_csv("Location_NY.csv",mangle_dupe_cols=False)

In [174]:
Initial_station = list(ShakeShack_area_station[ShakeShack_area_station["name"].isin(Location_NY[Location_NY["open_date"]=="2014Q4"]["name"])]\
["field_1"].unique())
Initial_station.extend(list(ShakeShack_area_station[ShakeShack_area_station["name"].isin(Location_NY[Location_NY["open_date"]=="2014Q4"]["name"])]\
["field_2"].unique()))
Initial_station
Initial_station = list(set(Initial_station))

In [100]:
#filtrate new opening stores out of New York after 2015

Location_non_NY_2015 = pd.DataFrame(Record[Record["Location"].str.contains("NY")==False])

len(Location_non_NY_2015)

249

In [102]:
#statistics of new opening stores in New York after 2015
Location_NY.groupby(["open_date"]).count()

,name
open_date,
2014-12-31,20
2015-Fourth,1
2016-Second,2
2016-Third,1
2017-Fourth,1
2017-Second,1
2017-Third,1
2018-Fourth,2
2018-Second,1


In [662]:
#statistics of new opening stores out of New York after 2015
Location_non_NY_2015.groupby(['Year','Quarter']).count()

Location
Year Quarter          
2015 First           2
     Fourth          7
     Second          8
     Third           3
2016 First           4
     Fourth         10
     Second          7
     Third           9
2017 First          13
     Fourth         14
     Second          7
     Third           8
2018 First           9
     Fourth         17
     Second         10
     Third           7
2019 First          11
     Fourth         24
     Second         18
     Third          17
2020 First          11
     Fourth         11
     Second          5
     Third           9
2021 First          10